In [15]:
#DNN

import tensorflow as tf
from tensorflow.contrib.layers import batch_norm
from tensorflow.contrib.layers import fully_connected

tf.reset_default_graph()

n_inputs=28*28
n_hidden1=300
n_hidden2=100
n_outputs=10
X=tf.placeholder(tf.float32,shape=(None,n_inputs),name='X')
#1.梯度消失或爆炸；难于训练；容易过拟合

#梯度爆炸易发生于RNN
#在sigmold函数和正态随机初始化方法下，每层输出的方差要远远大于输入的方差。在向前传播的过程
#中，方差持续增大，而logisitc函数的均值为0.5。输入越大，logistic的梯度越小。因此在最高层，
#梯度已然非常小，在反向传播过程中越来越小直至消失。解决的办法是让输出的方差与输入的方差相等
#梯度反向传播过程中的方差也不变，Xavier/Glorot initialization方法可在一定程度上实现这种可能。
#ReLU 与初始化策略相结合称为He initialization  P.278


#fully_connected()默认为Xavier（均匀分布）初始化。
he_init=tf.contrib.layers.variance_scaling_initializer()#He初始化，可更改mode="FAN-AVG"
hidden1=fully_connected(X,n_hidden1,weights_initializer=he_init,scope='h1')

he_init = tf.contrib.layers.variance_scaling_initializer()
hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu,
                          kernel_initializer=he_init, name="hidden1")
##书中使用tensorflow.contrib.layers.fully_connected()而不是tf.layers.dense()（本章编写时不存在）。 
##现在最好使用tf.layers.dense()，因为contrib模块中的任何内容可能会更改或删除
#dense()函数几乎与fully_connected()函数完全相同。 与本章有关的主要差异是：
###scope变成name，activation_fn变成activation（类似地，_fn后缀从诸如normalizer_fn之类的其他参数中移除），
###weights_initializer变成kernel_initializer等等。默认activation现在是None，而不是tf.nn.relu。 
###它不支持tensorflow.contrib.framework.arg_scope(),它不支持正则化的参数（稍后在第 11 章介绍）。




#fan-in /fan-out输入/输出连接的数目，He初始化只考虑fan-in，并不是Xavier那样的的fan-in
#和fan-out的均值。

#非饱和激活函数
#elu()
hidden1=fully_connected(X,n_hidden1,activation_fn=tf.nn.elu)

#leaky ReLU
def leaky_relu(z,name=None):
    return tf.maximum(0.01*z,z,name=name)
hidden1=fully_connected(X,n_hidden1,activation_fn=leaky_relu)

#批正则化 batch normalization

###用以解决梯度消失或爆炸，因为在训练过程中，由于前一层的参数发送变化，每层的输入的分布
#会变化变化，称为内部协变量漂移internal covariate shift
#在每层的激活函数前增加输入的标准化操作，增加尺度和漂移参数去标准化结果。即该操作迫使模型
#学习每层的最优规模和输入均值。该操作要求每次都要计算mini-batch的均值和标准差
 #P.282
#在测试阶段，则用整个训练集去计算均值和标准差，在训练阶段则用移动平均方法。因此每个批正则
#化层需要进行4个参数的学习：γscale,βoffset,μmean,σstandard deviation.

#批正则化可以大幅缓解梯度消失或爆炸问题，甚至可以用饱和激活函数，对初始化权值也不那么敏感，
#甚至可以设较大的学习率，同时降低对其他正则化技术的需求。

#批正则化省去了输入数据的正则化，但仍增加了模型的复杂度。因此，从计算速度角度考虑，需权衡
#ELU+He初始化与批正则化的表现差异。


is_training=tf.placeholder(tf.bool,shape=(),name='is_training')
#是否在训练阶段；如是进行mini-batch正则化；否则对全测试集正则化。
bn_params={'is_training':is_training,'decay':0.99,'updates_collections':None}
#正则化的参数；算法以指数递减的方法计算运行平均running averages;即学习率的衰减参数
#updates_collections控制是否在批正则化前更新running averages。
hidden1=fully_connected(X,n_hidden1,scope='hidden1',normalizer_fn=batch_norm,
                       normalizer_params=bn_params)
hidden2=fully_connected(hidden1,n_hidden2,scope='hidden2',normalizer_fn=batch_norm,
                       normalizer_params=bn_params)
logits=fully_connected(hidden2,n_outputs,activation_fn=None,scope='output',
                      normalizer_fn=batch_norm,normalizer_params=bn_params)
#ReLU函数无需进行scale;但其他激活函数需在bn_params设定scale=True

#条件域简化参数
with tf.contrib.framework.arg_scope(
[fully_connected],
normalizer_fn=batch_norm,
normalizer_params=bn_params):
    hidden1=fully_connected(X,n_hidden1,scope='hidden12')
    hidden2=fully_connected(hidden1,n_hidden2,scope='hidden22')
    logits=fully_connected(hidden2,n_outputs,scope='outputs2',activation_fn=None)
    
#梯度修剪
##进行梯度修剪，使梯度值不超过阈值以防止梯度爆炸。梯度修剪已为批正则化替代。
##优化器的minimize函数会先计算随后应用梯度，因此为应用梯度修剪需先调用cimpute_gradients()
##然后调用clip_by_value函数修剪梯度，最后调用apply_gradients
threshold=1.0
optimizer=tf.train.GradientDescentOptimizer(learning_rate)
grads_and_vars=optimizer.compute_gradients(loss)
capped_gvs=[(tf.clip_by_value(grad,-threshold,threshold),var) for grad,var in grads_and_vars]
training_op=optimizer.apply_gradients(capped_gvs)

#复用预训练层
##利用低层的网络参数进行迁移学习，而不是从头开始训练。输入特征相似的数据才能进行迁移学习。



## 非饱和激活函数
###ReLU的良好表现来源于其对正值的非饱和性。但并非完美
#dying ReLU:在训练中，除了0之外没有输出。有时候会出现一半的ReLU失效，尤其是当学习率设的
比较高的时候。当神经元权值更新，输入的加权和小于0时，神经元只能输出0.

### leaky ReLU
漏斗ReLU_{\alpha}=max{\alpha z,z},alpha通常设为0.01，长久休眠，但有机会苏醒。
#leaky ReLU的表现比ReLU好，并且alpha较大比较小好。

### random leaky ReLU
,alpha是在训练时随机挑选，在测试时固定在平均水平。

### parametric leaky ReLU，
alpha变成需要训练的参数。在大数据中表现优异，但在小数据中容易过拟合。

### exponetional linear unit (ELU) 
$ELU_{\alpha}(z)=\begin{cases}\alpha(exp(z)-1)&\ if\ z<0\\z&if\ z\ge 0\end{cases}$<br/>
在小于0处为负，保证输出均值接近0；在小于0时，梯度不为0；处处平滑，可加速梯度下降。但计算要更慢，训练也更慢。<br/>
一般选择：ELU>leaky ReLU>ReLU>tanh>logistic


In [ ]:
#在完成模型设计后，运行模型的时机由batch_norm层控制，需设定is_training占位符True或False
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for X_batch,y_batch in zip(X_batches,y_batches):
            sess.run(training_op,feed_dict={is_training:True,X:batch,y:y_batch})
        accracy_score=accuracy.eval(
        feed_dict={is_training:False,X:X_test_scaled,y:y_test})
        print(accuracy_score)

#复用TensorFlow模型
##复用全部模型
with tf.Session() as sess:
    saver.restore(sess,'./my_original_model.ckpt')
##仅复用一些变量的子集,如只复用hidden1,2,3
init=tf.global_variables_initializer()
reuse_vars=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='hidden[123]')
reuse_vars_dict=dict([(var.name,var.name) for var in reuse_vars])
original_saver=tf.Saver(reuse_vars_dict)#saver只存储与原模型对应的部分变量
new_saver=tf.Saver()#存储新模型

with tf.Session() as sess:
    sess.run(init)
    original_saver.restore('./my_orignal_model.ckpt')#从原模型重用hidden1,2,3
    '''....'''#完成训练
    new.saver.save('./my_new_model.ckpt')#存储新模型
    
    
#复用其他框架的模型
##如果模型是以其他框架写成的（如Theano），则只能手动指定
original_w=[...]#从其他框架加载模型参数
original_b=[...]#
X=tf.placeholder(tf.float32,shape=(None,n_inputs),name="X")
hidden1=fully_connected(X,n_hidden1,scope='hidden1')
[...]#创建模型的其他部分
#处理fully_connected创建的变量
with tf.variable.scope('',default_name='',reuse=True):#根命名域
    hidden1_weights=tf.get_variable('hidden1/weights')
    hidden1_biases=tf.get_variable('hidden1/biases')
    
#创建节点并初始化
original_weights=tf.placeholder(tf.float32,shape=(n_inputs,n_hidden1))
original_biases=tf.placeholder(tf.float32,shape=(n_hidden1))
assign_hidden1_weights=tf.assign(hidden1_weights,original_weights)
assign_hidden1_biases=tf.assign(hidden1_biases,original_biases)
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    sess.run(assign_hidden1_weights,feed_dict={original_weights:original_w})
    sess.run(assign_hidden1_biases,feed_dict={original_biases:original_b})


    

#封存（freezing） 低层网络
##底层网络用于获得底层特征，而底层特征是训练的基础，因此复用底层网络非常有用，当底层网络的
##权重确定，高层网络的训练会非常容易进行。封存底层网络最好的办法就是将要训练的但不包括底层
#网络的变量传递给优化器：
train_vars=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='hidden[34]outputs')
training_op=optimizer.minimize(loss,var_list=train_vars)#1,2层称为封存层


#缓存封存层
##由于封存层的权重不会改变，因此对于每个训练样本，可以缓存出最高的封存层的输出，训练过程要
##遍历整个样本很多次，而缓存封存层的方法可以大大提高训练速度，因为对于每个样本只需遍历一次。

hidden2_outputs=sess.run(hidden2,feed_dict={X:X_train})#用整个样本集训练底层网络

import numpy as np
n_epochs=100
n_batches=500
for epoch in range(n_epochs):
    shuffled_idx=rnd.permutation(len(hidden2_ouputs))
    hidden2_batches=np.array_split(hidden2_outputs[shuffled_idx],n_batches)
    #不在用训练样例的batch,而是底层网络输出的batch
    y_batches=np.array_split(y_train[shuffled_idx],n_batches)
    for hidden2_batch,y_batch in zip(hidden2_batches,y_batches):
        sess.run(training_op,feed_dict={hidden2:hidden2_batch,y:y_batch})
    
#更快的优化器

##Momentum优化器
optimizer=tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)

##NMG/NAG
optimizer=tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9,
                                     use_nesterov=True)
##AdaGrad

##RMSProp
optimizer=tf.train.RMSPropOptimizer(learning_rate=learning_rate,
                                    momentum=0.9,decay=0.9,epsilon=1e-10)
##Adam
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)

##FTRL系数模型优化
FTRLOptimizer(learning_rate=learning_rate)



##学习率调整

initial_learning_rate=0.1
dacay_steps=10000
dacay_rate=1/10
global_step=tf.Variable(0,trainable=False)
learning_rate=tf.train.exponential_decay(initial_learning_rate,global_step,dacay_steps,
                                        dacay_rate)
optimizer=tf.train.MomentumOptimizer(learning_rate,momentum=0.9)
training_op=optimizer.minimize(loss,global_step=global_step)

#其中，global_step用于记录当前迭代次数。然后调用exponential_decay函数并调用指数衰减学习率
#eta_0=0.1,r=10000,然后创建优化器MomentumOptimizer，调用衰减学习率。
##AdaGrad,RMSProp,Adam优化器是自适应学习率算法，不需要进行学习率调整。对其他优化器，指数
#和表现调整可以很快收敛。

#正则化

##L_1和L_2正则化
base_loss=tf.reduce_mean(xentropy,name='avg_xentropy')
reg_losses=tf.reduce_sum(tf.abs(weights1))+tf.reduce_sum(tf.abs(weights2))
loss=tf.add(base_loss,scale*reg_losses,name='loss')
#当网络层数较多时，这种方法不是很方便。Tensorflow中再创建变量时提供了*_regularizer选项，
##如l1_regularizer(),l2_regularizer,l1_l2_regularizer():
with arg_scope([fully_connected],
    weights_regularizer=tf.contrib.layers.l1_regularizer(scale=0.01)):
    hidden1=fully_connected(X,n_hidden1,scope='hidden1')
    hidden2=fully_connected(hidden1,n_hidden2,scope='hidden2')
    logits=fully_connected(hidden2,n_outputs,activation_fn=None,scope='out')

    
reg_loss=tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
loss=tf.add_n([base_loss]+reg_losses,name='loss')


#Dropout
#在训练阶段，随机地让一些隐层失活。
#提高dropout_rate可以防止过拟合
from tensorflow.contrib.layers import dropout
[...]
is_training=tf.placeholder(tf.bool,shape=(),name='is_training')
keep_prob=0.5
X_drop=dropout(X,keep_prob,is_training=is_training)
hidden1=fully_connected(X_drop,n_hidden1,scope='hidden1')
hidden1_drop=dropout(hidden1,keep_prob,is_training=is_training)
hidden2=fully_connected(hidden1_1,n_hidden2,scope='hidden2')
hidden2_drop=dropout(hidden2,keep_prob,is_training=is_training)
logits=fully_connected(hidden2_drop,n_outputs,activation_fn=None,scope='outputs')

#Max-Norm正则化
##对于任意一个神经元，限制其权值矩阵的L_2范数小于r,其中r为Max-norm超参数。实现的方法是
##：w<-w*r/L_2(w),减小r可以防止过拟合，也可以缓解梯度消失爆炸之谜。tensorflow没有现成的
##max-norm正则化工具，但可以以clip_weights进行

threshold=1.0#即max-norm参数
clipped_weights=tf.clip_by_norm(weights,clip_norm=threshold,axes=1)
clip_weights=tf.assgin(weights,clipped_weights)
#每次训练中都要执行该操作
with tf.Session()as sess:
    [...]
    for epoch in range(n_epochs):
        [...]
        for X_batch,y_batch in zip(X_batches,y_batches):
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
            clip_weights.eval()
#在模型设计阶段，还需连接每一层的weights参数。
hidden1=fully_connected(X,n_hidden1,scope='hidden1')
with tf.variable_scope('hidden1',reuse=True):
    weight1=tf.get_variable('weights')
#或者用根命名域
hidden1=fully_connected(X,n_hidden1,scope='hidden1')
hidden2=fully_connected(hidden1,n_hidden2,scope='hidden2')
[...]
with tf.variable_scope('',default_name='',reuse=True):#根命名域
    weights1=tf.get_variable('hidden1/weights')
    weights2=tf.get_variable('hidden2/weights')
#如果变量的名称，你可以以tensorboard，或者调用global_variables()函数打印出变量的名字
for variable in tf.global_variables():
    print(variable.name)
    

    
    #创建max_norm_regularizer()函数
def max_norm_regularizer(threshold,axes=1,name='max_norm',collection='max_norm'):
    def max_norm(weights):
        clipped=tf.clip_by_norm(weights,clip_norm=threshold,axes=axes)
        clip_weights=tf.assign(weights,clipped,name=name)
        tf.add_to_collection(collection,clip_weights)
        return None#没有正则损失项
    return max_norm
#然后就可以像调用其他正则器调用该函数
max_norm_reg=max_norm_regularizer(threshold=1.0)
hidden1=fully_connected(X,n_hidden1,scope='hidden1',weights_regularizer=max_norm_reg)
#但仍需要每次训练都调用clip_weights操作，因此需要将clip_weights节点加入max_norm 修剪操作
#的集合中，需要将修剪操作取出并每次运行之
#
clip_all_weights=tf.get_collection('max_norm')
with tf.Session()as sess:
    [...]
    for epoch in range(n_epochs):
        [...]
        for X_batch,y_batch in zip(X_batches,y_batches):
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
            sess.run(clip_all_weights)
            

## 初始化策略
<img src='https://raw.githubusercontent.com/apachecn/hands_on_Ml_with_Sklearn_and_TF/dev/images/chapter_11/t-11-1.jpg' width='600'>

##  调整，丢弃，替换高层网络
##对于新的训练任务，输出层与高层通常是无用的，因此通常需要调整和替换。一般的步骤是：
###封存所有复用层，训练观察其表现，然后从高层开始解封，通过反向传播调整高层网络。
##如果表现不好，而训练样本也不多，则尝试将高层网络丢弃并封存余下的隐层，然后迭代进行
##这个过程直至得的最优的复用网络数目。若训练样本足够多，那么你可以尝试替换隐层网络而非
##丢弃他们，甚至还可以增加隐层。


##  Model Zoos
##https://github.com/tensorflow/models
##https://github.com/ethereon/caffe-tensorflow


##  无监督预训练
##用受限玻尔兹曼机和自编码网络对网络进行无监督的预训练，然后利用有标签数据进行训练。 
##在梯度消失问题得到缓解之后，无监督预训练才渐渐被反向传播替代。但基于子自编码网络的
##预训练仍是一个有效的训练手段，尤其是当没有可复用的模型以及有标签的数据太少时。


## 作为辅助任务的预训练
##训练深度神经网络的最后一个方法是先训练一个易获得标签的、辅助任务的神经网络，然后复用
##底层网络。例如，建立人脸识别系统问题，你不大可能一开始就拥有足够的样本，但你可以在网络
##上搜集大量人脸图片，然后训练深度神经网络以识别任意两个图片是否是同一个人。然后复用其底层网络，建立自己的人脸识别系统。

##将所有训练样本标记为‘好’，然后生成‘坏’样本，训练网络以识别二者。
##另一种方法是训练神经网络对每个训练样本进行评分，用损失函数确保‘好’样本的得分高于‘坏’
###样本至少一个边际，称之为最大边际学习。



<img src='https://raw.githubusercontent.com/hzg0601/python/master/unsupervised_pretraining.png' widht='600'>

##  更快的优化器
##加速训练的四种方法：Xavier/Glorot、He初始化策略；非饱和激活函数；批标准化；重用预训练的网络。
  
###另一种重要的方法使用更快的优化器：Momentum,Nesterov Accelerated Gradient,AdaGrad,
###RMSProp,Adam,但通常情况下，最常用的优化器是AdamOptimizer，AdamOptimizer有三个超参数
###需要优化，但使用默认参数效果已经非常好了。

### Momentum优化器
如保龄球沿着光滑而轻微倾斜的表面运动那样，开始非常缓慢，而一旦获得足够的动量，速度就好非常快。梯度优化的原则是：$\theta=\theta-\eta \Delta_{\theta}J(\theta)$,它并不关心初始的梯度是多少，如果初始梯度很小，那么训练速度回非常慢。<br/>
  动量优化则不同，它的速度取决于前期的梯度是多少：每一次迭代中，它将局部梯度乘以学习率加入动量向量$\boldsymbol{m}$中，然后以前一个参数减去动量向量更新参数。换句话说，梯度作用于加度数而非速度。为防止动量变得过大，引入超参数$\beta$，简称为动量，默认值为0.9
$$\boldsymbol{m}\leftarrow\beta \boldsymbol{m}+\eta\Delta_{\theta}J(\theta)$$
$$\theta\leftarrow\theta-\boldsymbol{m}$$
如果梯度值为常数，那么最终速度是梯度乘以学习率再乘以$\frac{1}{1-\beta}$,动量优化器可以使训练加速逃离平缓期。在不使用批标准化的深度网络中，高层网络通常会有不同的尺度，此时应用动量优化器将非常有效。<br/>
如果没有摩擦系数$\beta$，动量优化会在最优值附近震荡。

### Nesterov Accelerated(Momentum) Gradient
NAG/NMG的思路是：不是在原位置而是动量方向稍微向前的方向测度损失函数的梯度。这可以显著提高训练速度，原因在于动量向量通常会朝向最优的方向。
$$\boldsymbol{m}\leftarrow\beta \boldsymbol{m}+\eta\Delta_{\theta}J(\theta+\beta\boldsymbol{m})$$
$$\theta\leftarrow\theta-\boldsymbol{m}$$

### AdaGrad
如果算法能够提前侦知梯度曲线的斜率结构，如陡峭的部分之后是平坦的部分，那么速度可以进一步提升。AdaGrad优化器可以沿最陡峭的维度缩小梯度向量。
$$\boldsymbol{s}\leftarrow\boldsymbol{s}+\Delta_{\theta}J(\theta)\otimes\Delta_{\theta}J(\theta)$$
$$\theta\leftarrow\theta-\eta\Delta_{\theta}J(\theta)\oslash\sqrt{\boldsymbol{s}+\epsilon}$$
第一步将梯度的平方累加到$\boldsymbol{s}$中，相当于$s_i\leftarrow s_i+(\partial/\partial \theta_iJ(\theta))^2$.如若损失函数的沿着第i维陡峭，那么$s_i$将越来越大。<br/>
第二步与梯度下降基本相同，唯一的不同在于，梯度向量按$\sqrt{\boldsymbol{s}+\epsilon}$倍缩小，$\oslash$代表按元素点除。$\epsilon$为防止除数为0而设的小数。<br/>
总而言之，这个算法的目的是为让学习率再训练中递减，它沿陡峭的维度收敛的比稍微倾斜的维度要快很多，也称为适应性学习率Adaptive learning rate.AdaGrad对二次型问题表现很好，但容易早停而陷入局部最优，因此不使用与深度网络。

### RMSProp
RMSProp通过开始时用指数衰减函数的方式只累加最近几次迭代的梯度以修正AdaGrad早停而陷入局部最优的缺陷。
$$\boldsymbol{s}\leftarrow\beta\boldsymbol{s}+(1-\beta)\Delta_{\theta}J(\theta)\otimes\Delta_{\theta}J(\theta)$$
$$\theta\leftarrow\theta-\eta\Delta_{\theta}J(\theta)\oslash\sqrt{\boldsymbol{s}+\epsilon}$$
在复杂问题中,RMSProp表现要优于AdaGrad.
### Adam优化器
Adam是自适应矩估计的代表，结合了Momentum和RMSProp的思路：它如Momentum一样追踪前向梯度的指数衰减平均值，如RMSProp一样追踪前向梯度平方的指数衰减平均值。
$$\boldsymbol{m}\leftarrow\beta_1\boldsymbol{m}+(1-\beta_1)\Delta_{\theta}J(\theta)$$
$$\boldsymbol{s}\leftarrow\beta_2\boldsymbol{s}+(1-beta_2)\Delta_{\theta}J(\theta)\otimes\Delta_{\theta}J(\theta)$$
$$\boldsymbol{m}\leftarrow\frac{\boldsymbol{m}}{1-\beta_1^T}$$
$$\boldsymbol{s}\leftarrow\frac{\boldsymbol{s}}{1-\beta^T_2}$$
$$\theta\leftarrow\theta-\eta\boldsymbol{m}\oslash\sqrt{\boldsymbol{s}+\epsilon}$$
其中，T代表迭代的次数。<br/>
1和2应用的指数衰减平均值，而不是指数衰减和。$\beta_1,beta_2$的初始值通常为0.9，和0.99，平滑项$\epsilon$通常为$10^{-8}$,$\eta$通常为0.001.<br/><br/>
以上所用的全部优化器都是基于一次偏微分的（Jacobians），但理论上也可以基于二次偏微分（Hessians）.但由于计算复杂度的问题而难以应用。<br/>
$l_1$正则化更易得的稀疏模型，但仅仅进行$l_1$正则化可能远远不够，因此需要Dual Averaging,称为Follow The Regularized Leader,Tensorflow提供FTRLOptimizer优化器实现FTRL-Proximal方法。


## 学习率调整
适应性学习率优化算法：AdaGrad,RMSProp,Adam.<br/>
leraning schedules包括：
### 先定分段常数学习率
先设定一个较大的学习率，然后规定一定epoches之后变为另一个学习率。
### 性能调整
每n次计算一次验证误差，当验证误差不再减少，以参数$\lambda$调整学习率。
### 指数调整
$\eta(t)=\eta_0 10^{-t/r}$,每10步调整一次学习率。
### 幂调整
$\eta{t}=\eta_0(1+t/r)^{-c}$,c通常设为1，比指数调整的缓慢些。<br/>
性能调整和指数调整的表现最好，但指数调整更易于执行，调参，并且收敛也稍快。




# 正则化
## 早停
在验证集表现下降的时候停止。设置一个区间，如每50次，更新一次表现更好的模型，记录运行的次数，一旦表现更好的模型保存超过一定的运行次数，如200次，就停止运行。
## $L_1$和$L_2$正则化
在损失函数中加入正则项即可。
## Dropout
## 数据增益
从现有数据中生成新数据，为数据增加**`可预测 `**的扰动。


Default | DNN configration
-|:-:|-:
Initialization | He initialization
Activation funtion  | ELU
Normalization | Batch Normalization
Regularizer | Dropout
Optimizer | Adam
Learning rate schedule | None

* 如果' 不能 '寻找到最优的学习率，则需加入学习率调整策略。
* 如果样本太少，可以引入数据增益。
* 如果想得到稀疏模型可以引入$L_1$正则化，如果想得到更稀疏的模型，可以引入FTRL优化器替代Adam优化器，并加入$L_1$正则项。
* 如果想获得快速实现的模型，则可以将批标准化步骤省去，将ELU替换为Leaky ReLU。

**学习率调整**

``` python

initial_learning_rate=0.1
dacay_steps=10000
dacay_rate=1/10
global_step=tf.Variable(0,trainable=False)
learning_rate=tf.train.exponential_decay(initial_learning_rate,global_step,dacay_steps,
                                        dacay_rate)
optimizer=tf.train.MomentumOptimizer(learning_rate,momentum=0.9)
training_op=optimizer.minimize(loss,global_step=global_step)

```

In [4]:
import tensorflow as tf
import numpy as np

he_init=tf.contrib.layers.variance_scaling_initializer()#He初始化，
hidden1=tf.layers.dense(X,n_hidden1,activation=tf.nn.relu,
                       kernel_initializer=he_init,name='hidden1')

#ELU非饱和函数
hidden1=tf.layers.dense(X,n_hidden1,activation=tf.nn.elu,name='hidden1')

def leaky_relu(z,name=None):
    return tf.maximum(0.01*z,z,name=name)
hidden1=tf.layers.dense(X,n_hidden1,activation=leaky_relu,name='hidden1')

In [6]:
#批标准化
import tensorflow as tf

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")

training = tf.placeholder_with_default(False, shape=(), name='training')

hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
bn1 = tf.layers.batch_normalization(hidden1, training=training, momentum=0.9)
bn1_act = tf.nn.elu(bn1)

hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
bn2 = tf.layers.batch_normalization(hidden2, training=training, momentum=0.9)
bn2_act = tf.nn.elu(bn2)

logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
logits = tf.layers.batch_normalization(logits_before_bn, training=training,
                                       momentum=0.9)
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
training = tf.placeholder_with_default(False, shape=(), name='training')


#使用partial（）函数简化代码

from functools import partial

my_batch_norm_layer = partial(tf.layers.batch_normalization,
                              training=training, momentum=0.9)

hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
bn1 = my_batch_norm_layer(hidden1)
bn1_act = tf.nn.elu(bn1)
hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
bn2 = my_batch_norm_layer(hidden2)
bn2_act = tf.nn.elu(bn2)
logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
logits = my_batch_norm_layer(logits_before_bn)

['GDN',
 'OPTIMIZER_CLS_NAMES',
 'OPTIMIZER_SUMMARIES',
 'RevBlock',
 'SPARSE_FEATURE_CROSS_DEFAULT_HASH_KEY',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_allowed_symbols',
 'apply_regularization',
 'avg_pool2d',
 'avg_pool3d',
 'batch_norm',
 'bias_add',
 'bow_encoder',
 'bucketized_column',
 'check_feature_columns',
 'conv2d',
 'conv2d_in_plane',
 'conv2d_transpose',
 'conv3d',
 'conv3d_transpose',
 'convolution2d',
 'convolution2d_in_plane',
 'convolution2d_transpose',
 'convolution3d',
 'convolution3d_transpose',
 'create_feature_spec_for_parsing',
 'crossed_column',
 'dense_to_sparse',
 'dropout',
 'elu',
 'embed_sequence',
 'embedding_column',
 'embedding_lookup_unique',
 'feature_column',
 'flatten',
 'fully_connected',
 'gdn',
 'group_norm',
 'images_to_sequence',
 'infer_real_valued_columns',
 'input_from_feature_columns',
 'instance_norm',
 'joint_weighted_sum_from_feature_columns',
 'l1_l2_

In [9]:
from functools import partial
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

if __name__ == '__main__':
    n_inputs = 28 * 28
    n_hidden1 = 300
    n_hidden2 = 100
    n_outputs = 10

    mnist = input_data.read_data_sets("/tmp/data/")

    batch_norm_momentum = 0.9
    learning_rate = 0.01

    X = tf.placeholder(tf.float32, shape=(None, n_inputs), name = 'X')
    y = tf.placeholder(tf.int64, shape=None, name = 'y')
    training = tf.placeholder_with_default(False, shape=(), name = 'training')#给Batch norm加一个placeholder

    with tf.name_scope("dnn"):
        he_init = tf.contrib.layers.variance_scaling_initializer()
        #对权重的初始化

        my_batch_norm_layer = partial(
            tf.layers.batch_normalization,
            training = training,
            momentum = batch_norm_momentum
        )

        my_dense_layer = partial(
            tf.layers.dense,
            kernel_initializer = he_init
        )#包含重复参数

        hidden1 = my_dense_layer(X ,n_hidden1 ,name = 'hidden1')
        bn1 = tf.nn.elu(my_batch_norm_layer(hidden1))
        hidden2 = my_dense_layer(bn1, n_hidden2, name = 'hidden2')
        bn2 = tf.nn.elu(my_batch_norm_layer(hidden2))
        logists_before_bn = my_dense_layer(bn2, n_outputs, name = 'outputs')
        logists = my_batch_norm_layer(logists_before_bn)

    with tf.name_scope('loss'):
        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits= logists)
        loss = tf.reduce_mean(xentropy, name = 'loss')

    with tf.name_scope('train'):
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        training_op = optimizer.minimize(loss)

    with tf.name_scope("eval"):
        correct = tf.nn.in_top_k(logists, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

    n_epoches = 20
    batch_size = 200
# 注意：由于我们使用的是 tf.layers.batch_normalization() 而不是 tf.contrib.layers.batch_norm()（如本书所述），
# 所以我们需要明确运行批量规范化所需的额外更新操作（sess.run([ training_op，extra_update_ops], ...)。
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

    with tf.Session() as sess:
        init.run()
        for epoch in range(n_epoches):
            for iteraton in range(mnist.train.num_examples//batch_size):
                X_batch, y_batch = mnist.train.next_batch(batch_size)
                sess.run([training_op,extra_update_ops],
                         feed_dict={training:True, X:X_batch, y:y_batch})
            accuracy_val = accuracy.eval(feed_dict= {X:mnist.test.images,
                                                    y:mnist.test.labels})
            print(epoch, 'Test accuracy:', accuracy_val)

In [ ]:
#也可以将train阶段改为，而替代sess.run([training_op,extra_update_ops]..)为sess.run(training_op,....)
with tf.name_scope('train'):
    optimizer=tf.train.GradientDescentOptimizer(learning_rate)
    extra_update_ops=tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(extra_update_ops):
        training_op=optimizer.minimizer(loss)
        
    

In [ ]:
#梯度修剪
threshold = 1.0

optimizer = tf.train.GradientDescentOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(loss)
capped_gvs = [(tf.clip_by_value(grad, -threshold, threshold), var)
              for grad, var in grads_and_vars]
training_op = optimizer.apply_gradients(capped_gvs)

In [ ]:
#复用tensorflow模型
n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_hidden3 = 50
n_hidden4 = 50
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4")
    hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=tf.nn.relu, name="hidden5")
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

learning_rate = 0.01
threshold = 1.0

optimizer = tf.train.GradientDescentOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(loss)
capped_gvs = [(tf.clip_by_value(grad, -threshold, threshold), var)
              for grad, var in grads_and_vars]
training_op = optimizer.apply_gradients(capped_gvs)

init = tf.global_variables_initializer()
saver = tf.train.Saver()


with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")

    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images,
                                                y: mnist.test.labels})
        print(epoch, "Test accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_new_model_final.ckpt")

In [ ]:
#只复用部分结果
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope="hidden[123]") # regular expression
reuse_vars_dict = dict([(var.op.name, var) for var in reuse_vars])
restore_saver = tf.train.Saver(reuse_vars_dict) # to restore layers 1-3

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_model_final.ckpt")

    for epoch in range(n_epochs):                                      # not shown in the book
        for iteration in range(mnist.train.num_examples // batch_size): # not shown
            X_batch, y_batch = mnist.train.next_batch(batch_size)      # not shown
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})  # not shown
        accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images,  # not shown
                                                y: mnist.test.labels}) # not shown
        print(epoch, "Test accuracy:", accuracy_val)                   # not shown

    save_path = saver.save(sess, "./my_new_model_final.ckpt")

In [ ]:
#学习率调整
 initial_learning_rate = 0.1
    decay_steps = 10000
    decay_rate = 1/10
    global_step = tf.Variable(0, trainable=False, name="global_step")
    learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step,
                                               decay_steps, decay_rate)
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9)
    training_op = optimizer.minimize(loss, global_step=global_step)

In [ ]:
n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
with tf.name_scope("train"):       # not shown in the book
    initial_learning_rate = 0.1
    decay_steps = 10000
    decay_rate = 1/10
    global_step = tf.Variable(0, trainable=False, name="global_step")
    learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step,
                                               decay_steps, decay_rate)
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9)
    training_op = optimizer.minimize(loss, global_step=global_step)
init = tf.global_variables_initializer()
saver = tf.train.Saver()
n_epochs = 5
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images,
                                                y: mnist.test.labels})
        print(epoch, "Test accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

In [ ]:
#L_1和L_2正则化
scale=0.01
my_dense_layer = partial(
    tf.layers.dense, activation=tf.nn.relu,
    kernel_regularizer=tf.contrib.layers.l1_regularizer(scale))

with tf.name_scope("dnn"):
    hidden1 = my_dense_layer(X, n_hidden1, name="hidden1")
    hidden2 = my_dense_layer(hidden1, n_hidden2, name="hidden2")
    logits = my_dense_layer(hidden2, n_outputs, activation=None,
                            name="outputs")

with tf.name_scope("loss"):                                     # not shown in the book
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(  # not shown
        labels=y, logits=logits)                                # not shown
    base_loss = tf.reduce_mean(xentropy, name="avg_xentropy")   # not shown
    reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    loss = tf.add_n([base_loss] + reg_losses, name="loss")

In [ ]:
#dropout
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")


training = tf.placeholder_with_default(False, shape=(), name='training')

dropout_rate = 0.5  # == 1 - keep_prob
X_drop = tf.layers.dropout(X, dropout_rate, training=training)

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X_drop, n_hidden1, activation=tf.nn.relu,
                              name="hidden1")
    hidden1_drop = tf.layers.dropout(hidden1, dropout_rate, training=training)
    hidden2 = tf.layers.dense(hidden1_drop, n_hidden2, activation=tf.nn.relu,
                              name="hidden2")
    hidden2_drop = tf.layers.dropout(hidden2, dropout_rate, training=training)
    logits = tf.layers.dense(hidden2_drop, n_outputs, name="outputs")

In [ ]:
#最大范数正则化
threshold = 1.0
weights = tf.get_default_graph().get_tensor_by_name("hidden1/kernel:0")
clipped_weights = tf.clip_by_norm(weights, clip_norm=threshold, axes=1)
clip_weights = tf.assign(weights, clipped_weights)

weights2 = tf.get_default_graph().get_tensor_by_name("hidden2/kernel:0")
clipped_weights2 = tf.clip_by_norm(weights2, clip_norm=threshold, axes=1)
clip_weights2 = tf.assign(weights2, clipped_weights2)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
def max_norm_regularizer(threshold, axes=1, name="max_norm",
                         collection="max_norm"):
    def max_norm(weights):
        clipped = tf.clip_by_norm(weights, clip_norm=threshold, axes=axes)
        clip_weights = tf.assign(weights, clipped, name=name)
        tf.add_to_collection(collection, clip_weights)
        return None # there is no regularization loss term
    return max_norm

max_norm_reg = max_norm_regularizer(threshold=1.0)
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu,
                              kernel_regularizer=max_norm_reg, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu,
                              kernel_regularizer=max_norm_reg, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    
n_epochs = 20
batch_size = 50
clip_all_weights = tf.get_collection("max_norm")

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            sess.run(clip_all_weights)
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images,     # not shown in the book
                                            y: mnist.test.labels})    # not shown
        print(epoch, "Test accuracy:", acc_test)                      # not shown

    save_path = saver.save(sess, "./my_model_final.ckpt")             # not shown

In [2]:
from scipy.io import loadmat
mnist_raw=loadmat('C:\\Users\\hzg0601\\datasets\\mnist-original.mat')
mnist = {
        "data": mnist_raw["data"].T,
        "target": mnist_raw["label"][0],
        "COL_NAMES": ["label", "data"],
        "DESCR": "mldata.org dataset: mnist-original",
    }
